In [113]:
import json
import pandas as pd

userList = [{"username": 'userName', "email": 'email'}]

listString = json.dumps(userList)

def foo(jsonstring):
  lst = json.loads(jsonstring)
  lst[0]["username"] = "Constanza"
  lst[0]["email"] = "cybarrat@gmail.com"
  return lst

newList = foo(listString)

print(newList)

[{'username': 'Constanza', 'email': 'cybarrat@gmail.com'}]


In [114]:
def getColScoreGapOffense(scoreGap):
    #----------------------------------------------------------------------------------
    # scoreGap
    #	Losing big	<-3
    # 0>=	Tie/Losing small	>=-3
    # 7>=	Winning small	>0
	# Winning big	>7
    # Regresar escenario = la columna que corresponde al scoregap
    if int(scoreGap) > 7:
        return 'Winning Big'
    elif int(scoreGap) > 0:
        return 'Winning Small'
    elif int(scoreGap) >= -3:
        return 'Tie/Losing Small'
    else:
        return 'Losing Big'

def getColScoreGapDefense(scoreGap):
    #----------------------------------------------------------------------------------
    # scoreGap
    #	Losing big	<-3
    # 0>=	Tie/Losing small	>=-3
    # 7>=	Winning small	>0
	# Winning big	>7
    # Regresar escenario = la columna que corresponde al scoregap
    if int(scoreGap) > 7:
        return 'Winning Big'
    elif int(scoreGap) > 0:
        return 'Tie/Winning Small'
    elif int(scoreGap) >= -3:
        return 'Losing Small'
    else:
        return 'Losing Big'

In [115]:
def getScoreGap(team, scoreGap, year, side):
    carpeta = "/Users/mramosalarcon/Google Drive/Creatividad/DVOA " + year
    if side == "Offense":
        df_scoreGap = pd.read_csv(carpeta + "/" + year + " DVOA by Score Gap Offense.csv")
        _scoreGapCurrentTeam = df_scoreGap.query('Team == "' + team + '"')
        _scoreGapCurrentTeam = _scoreGapCurrentTeam[['Team', getColScoreGapOffense(scoreGap), getColScoreGapOffense(scoreGap) + ' Rank', 'Total DVOA']]
        _scoreGapCurrentTeam.to_json(r'/Users/mramosalarcon/Documents/CC/scoreGapOffense.json')
        return _scoreGapCurrentTeam
    else:
        df_scoreGap = pd.read_csv(carpeta + "/" + year + " DVOA by Score Gap Defense.csv")
        _scoreGapOpponent = df_scoreGap.query('Team == "' + team + '"')
        _scoreGapOpponent = _scoreGapOpponent[['Team', getColScoreGapDefense(-scoreGap), getColScoreGapDefense(-scoreGap) + ' Rank', 'Total DVOA']]
        _scoreGapOpponent.to_json(r'/Users/mramosalarcon/Documents/CC/scoreGapDefense.json')
        return _scoreGapOpponent



In [116]:
def retrieve(jsoncase):
  case = json.loads(jsoncase)
  scoreGap = case[0]["scoregap"]
  quarter = case[0]["quarter"]
  currentYard = case[0]["currentYard"]
  currentTeam = case[0]["currentTeam"]
  opponent = case[0]["opponent"]
  avgDVOA = case[0]["optional"][0]["avgDVOA"]

  year = case[0]["year"]
  carpeta = "/Users/mramosalarcon/Google Drive/Creatividad/DVOA " + year

  #=============================================================================================
  _scoreGapCurrentTeam = getScoreGap(currentTeam, int(scoreGap), year, "Offense")
  
  _scoreGapOpponent = getScoreGap(opponent, int(scoreGap), year, "Defense")
  #=============================================================================================

  df_quarter = pd.read_csv(carpeta + "/" + year + " DVOA by Quarters and Halves Offense.csv")
  _quarterCurrentTeam = df_quarter.query('Team == "' + currentTeam + '"')
  
  df_quarter = pd.read_csv(carpeta + "/" + year + " DVOA by Quarters and Halves Defense.csv")
  _quarterOpponent = df_quarter.query('Team == "' + opponent + '"')

  df_down = pd.read_csv(carpeta + "/" + year + " DVOA by Down and Distance Offense.csv")
  _downCurentTeam = df_down.query('Team == "' + currentTeam + '"')
  
  df_down = pd.read_csv(carpeta + "/" + year + " DVOA by Down and Distance Defense.csv")
  _downOpponent = df_down.query('Team == "' + opponent + '"')

  df_yard = pd.read_csv(carpeta + "/" + year + " DVOA by Field Zone Offense.csv")
  _yardCurrentTeam = df_yard.query('Team == "' + currentTeam + '"')

  df_yard = pd.read_csv(carpeta + "/" + year + " DVOA by Field Zone Defense.csv")
  _yardOpponent = df_yard.query('Team == "' + opponent + '"')

  df_DVOA = pd.read_csv(carpeta + "/" + year + " Team DVOA Ratings Offense.csv")
  _DVOACurrentTeam = df_DVOA.query('Team == "' + currentTeam + '"')

  df_DVOA = pd.read_csv(carpeta + "/" + year + " Team DVOA Ratings Defense.csv")
  _DVOAOpponent = df_DVOA.query('Team == "' + opponent + '"')

  return _scoreGapOpponent



In [117]:
case = [{
    "scoregap": '3', 
    "quarter": '4', 
    "timeleft": '7', 
    "currentDown": '2', 
    "yardsLeft1stDown": '6', 
    "home": '1', 
    "currentYard": '-40', 
    "opponent": 'DAL', 
    "currentTeam": 'DEN', 
    "year": '2019', 
    "optional": [{
        "success": '1',
        "successFailReason": 'Test',
        "avgDVOA": '80'
        }]
    }]
caseList = json.dumps(case)
casesRetrieved = retrieve(caseList)
casesRetrieved.head()

,Team,Losing Small,Losing Small Rank,Total DVOA
18,DAL,-0.70%,17,3.00%


In [100]:
df_2019 = pd.read_csv("/Users/mramosalarcon/Google Drive/Creatividad/DVOA 2019/2019 DVOA by Score Gap Offense.csv")
df_2019.head()

,Team,Losing Big Rank,Losing Big,Tie/Losing Small Rank,Tie/Losing Small,Winning Small Rank,Winning Small,Winning Big Rank,Winning Big,Late & Close Rank,Late & Close,Total DVOA Rank,Total DVOA,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,BAL,6,25.80%,1,32.30%,6,24.40%,4,26.20%,11,8.50%,1,27.70%,NaN,NaN,NaN,NaN
1,DAL,5,29.10%,9,12.10%,1,43.30%,3,28.50%,1,44.60%,2,24.20%,NaN,NaN,NaN,NaN
2,KC,2,45.80%,3,21.80%,5,25.50%,7,18.80%,6,11.70%,3,22.80%,NaN,NaN,NaN,NaN
3,NO,3,40.40%,11,6.00%,4,26.70%,2,32.80%,3,26.60%,4,21.40%,NaN,NaN,NaN,NaN
4,SEA,12,10.30%,4,21.50%,2,36.40%,24,-15.70%,4,26.50%,5,17.10%,NaN,NaN,NaN,scoregap


In [101]:
_offenseCurrentTeam = df_2019.query('Team == "DEN"')
_offenseCurrentTeam.head()

,Team,Losing Big Rank,Losing Big,Tie/Losing Small Rank,Tie/Losing Small,Winning Small Rank,Winning Small,Winning Big Rank,Winning Big,Late & Close Rank,Late & Close,Total DVOA Rank,Total DVOA,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
25,DEN,24,-13.50%,21,-6.00%,11,3.90%,31,-33.60%,19,-6.70%,26,-10.80%,NaN,NaN,NaN,NaN
